In [1]:
!pip install web3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 501 kB 6.5 MB/s 
     |████████████████████████████████| 82 kB 976 kB/s 
     |████████████████████████████████| 101 kB 11.8 MB/s 
     |████████████████████████████████| 1.1 MB 46.3 MB/s 
     |████████████████████████████████| 103 kB 59.7 MB/s 
     |████████████████████████████████| 144 kB 49.7 MB/s 
     |████████████████████████████████| 271 kB 58.2 MB/s 
     |████████████████████████████████| 94 kB 1.0 MB/s 
     |████████████████████████████████| 45 kB 3.6 MB/s 
     |████████████████████████████████| 236 kB 49.2 MB/s 
     |████████████████████████████████| 2.3 MB 40.5 MB/s 
     |████████████████████████████████| 1.6 MB 55.3 MB/s 
     |████████████████████████████████| 1.9 MB 38.3 MB/s 
  Created wheel for parsimonious: filename=parsimonious-0.8.1-py3-none-any.whl size=42723 sha256=2bbb29f98bce2cbb1782431a1eb6cb0fc311f72c89646e88942b82b20

In [3]:
!pip install py-solc-x 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install -U "web3[tester]"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 4.8 MB/s 
     |████████████████████████████████| 276 kB 20.4 MB/s 
     |████████████████████████████████| 43 kB 1.1 MB/s 
     |████████████████████████████████| 829 kB 63.1 MB/s 
  Created wheel for pyethash: filename=pyethash-0.1.27-cp37-cp37m-linux_x86_64.whl size=51987 sha256=60bcd2e53f975ecd5c612277f24c25b93321f0444a43d6bcf766b55f5d50900b
  Stored in directory: /root/.cache/pip/wheels/01/ce/8f/3604e244cd91a6852b37ca0ab6eada34106081b001bdf6d487
  Created wheel for pysha3: filename=pysha3-1.0.2-cp37-cp37m-linux_x86_64.whl size=133598 sha256=ff38c20bcfb6d3ca81ccab7da3317a999fa2b7b3842703a9a1a34bc5d6e281e5
  Stored in directory: /root/.cache/pip/wheels/0d/9e/bc/789fa0986c1fef30cafcc29da4dd07bc17ecba3fab78e27ed6
Successfully built pyethash pysha3
  Attempting uninstall: eth-hash
    Found existing installation: eth-hash 0.4.0
    Uninstallin

In [5]:
from solcx import install_solc
install_solc(version='v0.6.0')

Version('0.6.0')

In [11]:
from web3 import Web3
import json
w3 = Web3(Web3.HTTPProvider("https://ropsten.infura.io/v3/fc774f68da714285bc25ac5822af61a6"))
w3

In [57]:
from solcx import compile_source

res=compile_source("""
pragma solidity >=0.6.0 <0.7.0;
contract Crowdfunding {
    address public author;
    mapping(address => uint) public joined;
    uint constant Target = 0.8 ether;
    uint public endTime;
    uint public price = 0.2 ether;
    bool public closed = false;
    constructor() public {
        author = 0xA1D91D7b6Bd771c53eF7B7d12805368B46E52467;
        endTime = now + 30 days;
    }
    function getAuthor() public view returns (address) {
        return address(author);
    }

    function updatePrice() internal {
        uint rise = address(this).balance / 1 ether * 2 ether;
        price = 2 ether + rise;
    }
    receive() external payable {
        require(now < endTime && !closed , "众筹已结束");
        require(joined[msg.sender] == 0, "你已经参与过众筹");
        require(msg.value >= price, "出价太低了");
        joined[msg.sender] = msg.value;
        updatePrice();
    }
    function withdrawFund() external {
        require(msg.sender == author, "你不是作者");
        require(address(this).balance >= Target, "未达到众筹目标");
        closed = true;
        msg.sender.transfer(address(this).balance);
    }
    function withdraw() external {
        require(now > endTime, "还未到众筹结束时间");
        require(!closed, "众筹达标， 众筹资金已提取");
        require(Target > address(this).balance, "众筹达标，你没法提取资金");
        msg.sender.transfer(joined[msg.sender]);
    }
}
""",
output_values=['abi', 'bin'])
_, info = res.popitem()

In [58]:
bytecode = info['bin']
abi = info['abi']

In [59]:
# with open('hello.abi', 'r') as f:
#     abi = json.load(f)

# with open('hello.txt', 'r') as f:
#     code = f.read()

# code = code.replace("/n", "")
contract = w3.eth.contract(abi=abi,bytecode=bytecode) # 合约对象

In [60]:
acct = w3.eth.account.from_key("49c3a52465e2a66267da1fda7b1983dbbd49fa1730b2251f4e6b82ddfe77dd3a")
acct

In [61]:
construct_txn = contract.constructor().buildTransaction({
'from': acct.address,
'nonce': w3.eth.getTransactionCount(acct.address),
'gas': 5000000,
'gasPrice': w3.toWei('21', 'gwei')}) # 构造合约部署交易

signed = acct.signTransaction(construct_txn) # 交易签名
tx_id = w3.eth.sendRawTransaction(signed.rawTransaction)
print(tx_id.hex())

0xe34186cf5621ca5de8443de053c5524a54bfbf7e9c6ccf94c59b1e0bda7f1f00


In [62]:
contract_addr = w3.eth.waitForTransactionReceipt(tx_id.hex()).contractAddress # 通过交易 id 查询合约地址

In [63]:
contract_addr

'0x26460922f355FD59662c3DD1Bbd4dd1E010B824a'

In [64]:
c = w3.eth.contract(contract_addr, abi=abi)
c.all_functions() # 输出该合约可以调用的所有函数

[<Function author()>,
 <Function closed()>,
 <Function endTime()>,
 <Function getAuthor()>,
 <Function joined(address)>,
 <Function price()>,
 <Function withdraw()>,
 <Function withdrawFund()>]

In [65]:
c.functions.getAuthor().call()

'0xA1D91D7b6Bd771c53eF7B7d12805368B46E52467'